# General instructions for all labs

1. To turn in:
 - this python notebook, filled out (2 pts)
 - a *standalone* PDF report that contains all the plots, and the answers to all the discussion questions (2 pts)

2. Use of ChatGPT / CoPilot / etc:
   - Allowed, but you own everything that is generated
   - This means that any part of the solution can be asked in the quiz. It can be as detailed as "What was the batch size you used in training" or specific as "what exactly does masking do in this case?" Any discussion question is also game for a quiz question.
   - If I find AI usage to be excessive. I can individually drag any of you in for a 1-1 meeting, in which I grill you on your code. If it looks like irresponsible copy/pasting, without proper understanding, I reserve the right to drastically lower your grade, or even submit cases to GGAC for ethical review.
  
3. Use of peer collaboration:
   - In general not allowed. (Discussion / comparing answers is ok, but work on actual coding independently.)
   - Exceptions can be made if you all wrote your own training script, but 1. it takes forever to train or 2. you don't have great compute resources. Then you can share a trained model amongst yourself *and declare it on your pdf*. However, the code for training *still must be written by yourself*
     




# Lab 3: Scalable Retrieval with Project Gutenberg

Modern AI systems don’t just generate answers — they rely on **retrieval** to find the right context before generation. Whether you’re building a search engine, a recommendation system, or a Retrieval-Augmented Generation (RAG) pipeline, the quality and speed of retrieval often determine how useful the final system will be.

In this lab, you’ll explore the core ideas of **scalable** retrieval using Project Gutenberg texts. Books are long, hierarchical, and varied — which makes them an excellent testbed for learning how to index, search, and evaluate retrieval aton, or just keep it high-level so the reranker is an assignment step?
tudents can *see* the difference between the two structures?
letions spelled out as mini-exercises)?


## 📚 The Project Gutenberg Corpus

For this lab, we will be working with texts from **Project Gutenberg**, a large collection of public-domain books. Project Gutenberg has been digitizing and distributing literary works since 1971, making it one of the oldest and largest open digital libraries. The collection spans over **70,000 works**, ranging from classic literature to historical documents.

Because the raw Project Gutenberg site can be tricky to scrape (inconsistent file formats, encodings, and compression), we’ll use a cleaned dataset hosted on Kaggle:

👉 [Project Gutenberg – Over 70,000 Books (Kaggle Dataset)](https://www.kaggle.com/datasets/jasonheesanglee/gutenberg-over-70000)

This version provides a consolidated, reproducible collection of the corpus, which avoids many of the Unicode and file format errors students encounter when downloading directly from gutenberg.org.

We’ll use this dataset as our **source corpus** for chunking, embedding, and retrieval experiments.



## Part 1: Download and Inspect the Data

1. **Download the dataset**

   * Use the Kaggle link above: [Project Gutenberg – Over 70,000 Books](https://www.kaggle.com/datasets/jasonheesanglee/gutenberg-over-70000).
   * Unzip the dataset into a directory on your computer. You should see:

     * Many book files in `.pkl` format (each file contains the text of a book).
     * A metadata file called `gutenberg_over_70000_metadata.csv`.

2. **Inspect the book files and metadata**

   * Scripts are provided to walk through the directory and preview the contents of `.pkl` files (e.g., first 100 characters or tokens).
   * The metadata file contains details such as book ID, title, and author.
   * Run them and look around a bit.

3. **Load the data**
   * You will extend these inspection scripts to combine the book text and metadata into a single data structure:

     * **Key:** book number (ID)
     * **Value:** dictionary with both the text and metadata details.
    **You may not want to load all the data at once, as that is very memory intensive. Just have a natural break point at, say, 1000 books, which you can remove later once we decide to modify this function to be more efficient.**



In [ ]:
import os
import pickle
import pandas as pd

def load_metadata(root_dir):
    filepath = os.path.join(root_dir, 'gutenberg_over_70000_metadata.csv')
    try:
        df = pd.read_csv(filepath)
        
        print("\nFilepath:", filepath)
        print(df.head(10))          # Show first 10 rows
        print("Total rows:", len(df))
    
    except Exception as e:
        print(f"Error opening {filepath}: {e}")
    
def preview_pkl_files(root_dir):
    """
    Traverse root_dir and subdirectories, open .pkl files, 
    and print a preview of the first 100 characters or words.
    """
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".pkl"):
                filepath = os.path.join(subdir, file)
                try:
                    with open(filepath, "rb") as f:
                        data = pickle.load(f)
                    
                    print('\n',filepath, data[:10],len(data))
                
                except Exception as e:
                    print(f"Error opening {filepath}: {e}")

# Example usage

load_metadata('archive')
#preview_pkl_files("archive")




## 📖 From Text to Embeddings

Note that actually loading the raw text of all the books is extremely cumbersome, and also not that useful for retrieval. Instead, for retrieval systems, we don’t search directly over words — we search in **embedding space**, where each passage of text is represented as a dense vector.

Why do we need **chunking**?

* Embedding models have a **fixed input size** (typically 512–1024 tokens). A whole book is far too long to fit.
* Even if a model could process the entire book, a single embedding would dilute meaning — specific details would get lost.
* By splitting the book into smaller, overlapping passages (e.g. 500 words with 50 words overlap), we keep each chunk semantically coherent and within the model’s capacity.

Why do we need **aggregation**?

* If we only keep passage-level embeddings, retrieval becomes more expensive (millions of vectors).
* Sometimes we want a **compact representation** of the entire book (for catalog search, or as a coarse filter before drilling down).
* Aggregating embeddings — for example by **averaging** all chunk embeddings — provides this compact representation while still reflecting the book’s overall content.
* Averaging is a simple and widely used method: it balances the contributions of all chunks without blowing up dimensionality. Other methods (max pooling, weighted averaging, multi-vector representations) exist, but averaging is the most practical starting point.

---
--

Since entire books are far too long to embed directly, we need to:

1. **Chunk** the book into smaller passages (e.g. \~500 words each, with overlap).
2. **Embed** each passage using a pretrained model.
3. **Aggregate** the embeddings into a single book-level vector (e.g. by averaging).

Computing embeddings for all chunks can be expensive. To make this feasible, we will **subsample** by taking only the first *N* chunks (e.g. 000) per book. This reduces memory and runtime while still giving a representative eme students?



### Your Task

* First, look over the two functions offered to you below. You can use them, modify them, or rewrite your own functions. If you use these ones, please fill out the prompts in the comments, as you could be asked about it later.
* Next, modify your loading function from above to load the embedding computed from the text, rather than the entire text itself. You should notice that memory does not grow nearly as large.

Once it is all loaded, you can save it in a pkl file, to be easily loaded in the future.

This step can take a while. You may not want to do it last minute.


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def chunk_text(text, chunk_size=500, overlap=50, max_num_chunks = 10):
    """
    What does this function do?

    What is the input?

    What is the output?
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words) and len(chunks) < max_num_chunks:
        end = min(start + chunk_size, len(words))
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap  # slide window with overlap
    return chunks

def embed_book(text, chunk_size=500, overlap=50, aggregate="mean"):
    """
    What does this function do?

    What is the input?

    What is the output?
    """
    
    chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
    
    embeddings = model.encode(chunks, convert_to_numpy=True)
    
    return embeddings

"""
# --- Example usage ---
sample_text = books['167']['text']

chunk_embeddings = embed_book(sample_text)

print("Number of chunks:", len(chunk_embeddings))      # depends on chunk_size
print("First chunk embedding shape:", chunk_embeddings[0].shape)
"""

## Part 1 – Step 1: Build a Flat k-NN Graph

1. **Pairwise Distances**

   * In your notebook, write out the underlying equations for this computation.
   * Comment on the computational and memory complexity of this approach (it is $O(m^2)$ in both time and space, where $m$ is the number of books).
   * Make sure you understand the math — don’t just memorize it. I may ask you to reproduce it.

2. **Visualization with PCA**

   * Use PCA to reduce the embeddings to 2D, and plot the books in this space.
   * Instead of labeling points with IDs, annotate them with their book titles from the metadata. For example:

     ```python
     plt.text(embedding_2d[i, 0], embedding_2d[i, 1], book_embs[label]['metadata']['Book Title'], fontsize=6, alpha=0.7)
     ```
   * Comment on the geometry and clustering of these visual embeddings. Do they make sense?

3. **Nearest Neighbor Graph**

   * Create a nearest neighbor graph using `networkx`.
   * Each node corresponds to a book embedding.
   * You may connect nodes in one of two ways:

     * **Fixed degree (k-NN):** Connect each node to its $d$ closest neighbors, where $d$ is a hyperparameter (e.g. 5, 10, 20).
     * **Distance threshold:** Connect each node to any other node whose distance is less than a threshold $\varepsilon$, for example the 15th percentile of all pairwise distances.
   * Instead of plotting the raw graph with all its edges (which quickly becomes unreadable), visualize the graph structure using its adjacency matrix.

4. **Discussion**

   * Comment on the pros and cons of constructing an NN graph in each of the two ways.
   * How sensitive is the graph structure to the choice of $d$ or $\varepsilon$?
   * Compare how graph connectivity changes with the hyperparameter (clusters vs. isolated nodes).
 


## Part 1 Step 2. **Perform NN search over the graph**

1. Given a **query vector**, traverse the graph to find the nearest neighbors.  

2. First, perform **exact search**:  
   * Embed the query.  
   * Compute squared distances to all book embeddings.  
   * Find and print the 5 nearest neighbors.  
   * Use Google (or another source) to verify whether these books are actually related to your query summary.  

3. Next, try an **approximate search** using *NN descent*:  
   * Start from a random node.  
   * Look for the neighbor that is closest to the query node.  
   * Keep moving until you can’t find a closer neighbor.  
   * Return the path of visited nodes and the final result.  

4. **Discussion:**  
   * What is the computational complexity of finding the exact *d* neighbors for each query?  
   * How could approximate nearest neighbors reduce this complexity?  
   * What problems can occur with approximate NN (e.g., local optima)?  
   * How could we bypass these problems (e.g., by using more than one random seed)?  

5. **Visualization (optional):** Plot the query and its connected neighbors on your PCA projection of the embeddings.

```

# --- Query text ---
query = """In a near-future world where borders constantly shift due to climate change 
and political unrest, a young cartographer named Elara is recruited by an international 
coalition to create the first “living map” — a dynamic atlas that redraws itself in real time. 
As she works, she discovers that the map doesn’t just reflect the world, but begins to shape it: 
cities vanish from memory, coastlines retreat overnight, and entire communities are erased when 
they no longer appear on her charts. Torn between her duty to her employers and her conscience, 
Elara sets out on a journey across collapsing nations to uncover who is really controlling the 
map — and whether she can break the paradox before the world is redrawn beyond recognition."""

```



In [ ]:
import numpy as np
import random
import time, tracemalloc




def profile(fn, fn_name, *args, **kwargs):
    print('profile',fn_name)
    tracemalloc.start()
    t0 = time.perf_counter()
    result = fn(*args, **kwargs)   # run the function
    t1 = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print(f"{fn_name} took {t1 - t0:.4f}s, memory peak {peak/1e6:.2f} MB")
    return result


## Part 1 Step 3. **Add and Delete Nodes Online**

In real systems, data is not static — new documents (or books) arrive, and old ones may be removed. A good retrieval system must be able to **update its index online** without rebuilding everything from scratch.  

1. **Exact insertion and deletion**  
   * Insert a new node by computing its distance to **all existing nodes**.  
     - Find its nearest neighbors (e.g. top-k or within an ε-threshold).  
     - Connect it to those neighbors in the graph.  
   * Delete a node by removing it and repairing its neighbors.  
     - If nodes lose important edges (e.g. drop below degree k), recompute distances among affected neighbors to re-establish the k-NN property.  
   * **Question:** What is the computational complexity of inserting or deleting nodes *exactly* in this way? How does this scale as the graph grows?  

2. **Approximate insertion and deletion**  
   * Instead of comparing against all nodes:  
     - Start from a random entry node.  
     - Traverse the graph greedily, moving to neighbors closer to the new node.  
     - Use the visited nodes as candidates for edges.  
   * For deletion, simply remove the node and its edges without full repair, letting future approximate traversals fill in missing connections.  
   * **Question:** Where do the computational reductions come from in this approximate version? What trade-offs are introduced (e.g. possible missed neighbors, local optima)?  

3. **Discussion**  
   * Why is online add/delete functionality important in real retrieval systems?  
   * Compare the costs of exact vs approximate methods. In what scenarios would exact updates be necessary, and when is approximation acceptable?  


## Part 2: **Hierarchical graphs**


So far, you have built and experimented with a **flat nearest neighbor graph**.  
Now we will move to a more scalable structure: the **Hierarchical Navigable Small-World (HNSW) graph**.  

### 🔎 What is HNSW?
HNSW is a graph-based data structure for **approximate nearest neighbor (ANN) search**.  
It extends the idea of a flat k-NN graph into a **multi-level hierarchy**, inspired by skip lists:  
- Each element is assigned a *maximum layer* at random.  
- Higher layers are sparse (few nodes, long-range connections).  
- Lower layers are dense (many nodes, local connections).  
- Searching starts at the top, then proceeds down through the levels, refining at each stage.  

This combination of *random layering* and *graph connectivity* yields very efficient nearest neighbor search.



## Part 2 Step 1: Construction of the HNSW Graph
1. **Random Level Assignment**  
   * Each node is assigned a maximum level \( L \), sampled from an exponential distribution.  
   * Most nodes only appear at the lowest level, while a few appear in many levels (acting as "hubs").  

2. **Layered Graphs**  
   * For each level \( \ell \), connect nodes to their nearest neighbors (just like in the flat k-NN graph).  
   * Higher levels have fewer nodes, so connections there act like "express lanes."  
   * Lower levels are more detailed, ensuring accuracy.  

3. **Hierarchy**  
   * The result is a stack of graphs: top layers are sparse and global, bottom layers are dense and local.  



I have given you a backbone of an HNSW that includes the build function. Answer the following questions.

 - Which layers is the densist?
 - Which layer contains the entry point?
 - What does the method sample_level return for a node? What is m_L? at graph?  

In [ ]:
import numpy as np
import networkx as nx

class HNSW:
    def __init__(self, max_m=5, m_L=1.0, seed=42):
        """
        Simplified HNSW-style graph for teaching.
        """
        self.max_m = max_m
        self.m_L = m_L
        self.rng = np.random.default_rng(seed)
        self.layers = []             # list of graphs, one per layer
        self.node_levels = {}        # mapping: node_id -> max level
        self.embeddings = None       # store embeddings
        self.next_id = 0             # assign new IDs sequentially
        self.entry_point = None      # node ID at top level
        self.current_max_level = -1  # highest level index so far

    def sample_level(self):
        """Sample the maximum level for a node (exponential distribution)."""
        u = self.rng.random()
        return int(-np.log(u) * self.m_L)

    def build(self, embeddings):
        """Build HNSW from scratch."""
        m = embeddings.shape[0]
        self.embeddings = embeddings.copy()
        levels = [self.sample_level() for _ in range(m)]
        max_level = max(levels) if m > 0 else -1
        self.current_max_level = max_level

        # Create one graph per level
        self.layers = [nx.Graph() for _ in range(max_level + 1)]

        for i in range(m):
            self.node_levels[i] = levels[i]
            for L in range(levels[i] + 1):
                self.layers[L].add_node(i)

        # Connect nodes at each level
        for L in range(max_level + 1):
            nodes_at_L = [i for i in range(m) if levels[i] >= L]
            if len(nodes_at_L) <= 1:
                continue

            X = embeddings[nodes_at_L]  # shape (n_L, d)
            # Pairwise squared distances
            D = np.sum(X**2, axis=1)[:, None] + np.sum(X**2, axis=1)[None, :] - 2 * X @ X.T
            np.fill_diagonal(D, np.inf)

            nearest_matrix = np.argsort(D, axis=1)[:, :self.max_m]
            for row_idx, i in enumerate(nodes_at_L):
                for j_local in nearest_matrix[row_idx]:
                    j = nodes_at_L[j_local]
                    self.layers[L].add_edge(i, j, weight=float(D[row_idx, j_local]))

        # Choose entry point as any node at top level
        if m > 0:
            self.entry_point = max(range(m), key=lambda nid: levels[nid])
        else:
            self.entry_point = None

        self.next_id = m
        return self.layers, self.node_levels
 


## Part 2 step 2

Next, add the following class functions:

### hnsw.search
1. **Start from the top layer** (which has the fewest nodes).  
2. **Greedy search**: at each step, move to the neighbor that is closest to the query.  
3. When no closer neighbor exists, **drop down one level** and continue.  
4. At the lowest layer, refine until the nearest neighbor (or set of neighbors) is found.  

This ensures a fast global-to-local search process.

---

### hnsw.insert
1. **Sample a level** for the new node.  
2. **Search**:  
   * Start from the top layer and traverse greedily to find a close entry point.  
   * At each level down to the node’s maximum level, continue refining its nearest neighbors.  
3. **Connect**:  
   * Add the new node to each of its assigned layers.  
   * Link it to its nearest neighbors in those layers (ensuring max degree is not exceeded).  

This way, new nodes integrate into both global and local structures.

---

### hnsw.delete
1. **Remove the node** from all layers in which it appears.  
2. **Reconnect neighbors** if necessary:  
   * At each layer, consider the deleted node’s neighbors.  
   * Re-link them if their degree drops below the required minimum or connectivity is at risk.  
3. In practice, many implementations avoid explicit repair and rely on redundancy in the graph to maintain accuracy.  

---

### 📝 Your Task
- Extend your code to use an **HNSW-style hierarchical graph** instead of a flat one.  
- Practice **traversal, insertion, and deletion** in this setting.  
- Compare the **search paths** between flat graphs and HNSW graphs:  
  * How many nodes are visited?  
  * How does the path length differ?  
  * How does the random layer




### Questions for you

- **Layer assignment strategy:**  
  How are levels assigned to each node under this exponential scheme?  Is a node assigned to just one layer, or multiple layers? Explain the exact assignment scheme. 
  What are the advantages and disadvantages of this method compared to deterministic or uniform assignments?  

- **Computational and memory bottlenecks:**  
  Where do the main costs arise (construction, traversal, updates)?  
  At which stages are these bottlenecks reduced or avoided?  
  Why is this hierarchical design favored over a single flat graph?  



## Part 3: Scalable Retrieval with FAISS

Now that you’ve built your own HNSW structure, it’s time to see how it compares to industry-strength libraries. FAISS (Facebook AI Similarity Search) is a widely used toolkit for nearest neighbor retrieval at scale. In this part, you will explore FAISS’s **built-in indexing methods** and evaluate how they differ from your own implementation.

1. **Indexing Approaches**

   * **Flat Index (Exact Search):**

     ```python
     import faiss
     d = embeddings.shape[1]   # embedding dimension
     index_flat = faiss.IndexFlatL2(d)   # exact search
     index_flat.add(embeddings)          # add all vectors
     D, I = index_flat.search(embeddings[:5], k=5)  # search first 5 queries
     ```

   * **HNSW Index (Approximate Search):**

     ```python
     index_hnsw = faiss.IndexHNSWFlat(d, 32)  # 32 = max neighbors per node
     index_hnsw.hnsw.efConstruction = 40      # build-time parameter
     index_hnsw.add(embeddings)
     D_hnsw, I_hnsw = index_hnsw.search(embeddings[:5], k=5)
     ```

2. **Comparison**

   * **Computation:** How does query speed change between the flat index and FAISS HNSW as the dataset size grows?
   * **Memory:** How much extra memory is used to store the graph structure in FAISS HNSW compared to the flat index?
   * **Quality:** How close are the retrieved neighbors from FAISS HNSW to the ground-truth neighbors found by exact search? (Measurectly measure the speed difference?
and recommendation)?
